### Recommendations Rank & Knowledge Based: 


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [2]:
reviews.head(10)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,2,816711,8,1379963769,2013-09-23 19:16:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,2,1091191,7,1391173869,2014-01-31 13:11:09,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,2,1103275,7,1408192129,2014-08-16 12:28:49,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,2,1322269,7,1391529691,2014-02-04 16:01:31,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,2,1390411,8,1451374513,2015-12-29 07:35:13,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [3]:
movies.head(5)

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [4]:
reviews.shape

(712337, 23)

In [5]:
reviews.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp', 'date', 'month_1',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7',
       'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'year_2013',
       'year_2014', 'year_2015', 'year_2016', 'year_2017', 'year_2018'],
      dtype='object')

In [6]:
movies.shape

(31245, 35)

In [7]:
movies.columns

Index(['movie_id', 'movie', 'genre', 'date', '1800's', '1900's', '2000's',
       'History', 'News', 'Horror', 'Musical', 'Film-Noir', 'Mystery',
       'Adventure', 'Sport', 'War', 'Music', 'Reality-TV', 'Adult', 'Crime',
       'Family', 'Drama', 'Talk-Show', 'Biography', 'Sci-Fi', 'Fantasy',
       'Romance', 'Game-Show', 'Action', 'Documentary', 'Animation', 'Comedy',
       'Short', 'Western', 'Thriller'],
      dtype='object')

______________

_________

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [8]:
All_data = movies.merge(reviews.groupby(by=['movie_id']).mean().reset_index()[['rating', 'movie_id']],on='movie_id')
All_data = All_data.rename(columns={'rating': 'rating_avg'})
All_data = All_data.merge(reviews.groupby(by=['movie_id']).count().reset_index()[['rating', 'movie_id']],on='movie_id')
All_data = All_data.rename(columns={'rating': 'rating_count'})
All_data = All_data.merge(reviews.groupby(by=['movie_id']).min().reset_index()[['rating', 'movie_id']],on='movie_id')
All_data = All_data.rename(columns={'rating': 'rating_min'})
temp=reviews.groupby(by=['movie_id']).max().reset_index()
temp = temp.rename(columns={'date': 'last_review'})
All_data = All_data.merge(temp[['last_review', 'movie_id']],on='movie_id')
All_data = All_data.query("rating_count>=5").sort_values(by='rating_avg', ascending=False)
All_data = All_data.sort_values(by=['rating_avg', 'rating_count', 'last_review'], ascending=(False,False,False))



def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    
    return All_data[:n_top]['movie'].values.tolist() # a list of the n_top movies as recommended

In [9]:
All_data


,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Documentary,Animation,Comedy,Short,Western,Thriller,rating_avg,rating_count,rating_min,last_review
28837,4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,...,0,0,1,0,0,0,10.000000,48,10,2016-08-14 17:16:50
29398,5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,...,0,0,1,1,0,0,10.000000,28,10,2016-01-08 00:44:43
29894,5688932,Sorry to Bother You (2018),Comedy|Fantasy|Sci-Fi,2018,0,0,1,0,0,0,...,0,0,1,0,0,0,10.000000,14,10,2018-06-17 01:44:48
24083,2737018,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,...,0,0,0,0,0,0,10.000000,10,10,2015-05-10 22:56:01
23613,2560840,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,0,0,0,...,1,0,0,0,0,0,10.000000,6,10,2016-01-23 00:30:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911,60666,Manos: The Hands of Fate (1966),Horror,1966,0,1,0,0,0,1,...,0,0,0,0,0,0,1.857143,7,1,2018-05-06 19:18:10
25082,3108604,American Poltergeist (2015),Thriller,2015,0,0,1,0,0,0,...,0,0,0,0,0,1,1.800000,5,1,2017-10-31 03:02:32
25325,3187378,The Asian Connection (2016),Action,2016,0,0,1,0,0,0,...,0,0,0,0,0,0,1.666667,6,1,2017-07-11 06:13:16
28129,4458206,Kod Adi K.O.Z. (2015),Crime|Mystery,2015,0,0,1,0,0,0,...,0,0,0,0,0,0,1.000000,30,1,2018-01-20 04:44:12


Using the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [10]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1, 20)

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5)

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100)

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35)



In [11]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


_______________________

### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History', 'Comedy'])
```

In [12]:
def popular_recommendations_Filtered(user_id, n_top, years, genres):
    temp =All_data[ All_data.date.isin(years) | All_data.genre.isin(genres)]
    if temp.count()[0] > 0:
        return temp[:n_top]['movie'].values.tolist()
    else:
        return All_data[:n_top]['movie'].values.tolist()

In [13]:
popular_recommendations_Filtered(8, 10, [2013,1886], ['Drama','Comedy'])

['Selam (2013)',
 'Romeo Juliet (2009)',
 'Shijie (2004)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Kirik Party (2016)',
 'Blood Brother (2013)',
 'Poshter Girl (2016)',
 'Umberto D. (1952)']